# Quality Model App

## Import Requirements

In [ ]:
!pip install ipywidgets=7.7.3

In [1]:
import os
import io
import random
import hashlib
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image

## Specify Data Source

In [2]:
# Specify input directory as data source
# input_dir = '../dataset/subset2'
input_dir = '../dataset/Tile_Generator_Genetic_Algo_V1_16x16-2023-23-2--16-01-20'

## Run on Data Source

In [3]:
def create_input_data_dict(input_dir):

    # Placeholder for dict to contain result from running on data source.
    data_dict = {}

    print ('[INFO] Running on Data Source...')

    # Walking thru files
    for root, _, files in os.walk(input_dir):
        for file in files:
            # Get file path
            file_path = f'{root}/{file}'
            # Check if file is png or jpg
            if os.path.splitext(file_path)[-1] == '.png' or os.path.splitext(file_path)[-1] == '.jpg':
                try:
                    # Get image bytes
                    with open(file_path, 'rb') as img_file:
                        img_bytes = img_file.read()
                    # Compute hash
                    hasher = hashlib.sha256()
                    hasher.update(img_bytes)
                    hash_id = hasher.hexdigest()
                    data_dict[hash_id]={'file_path':file_path, 'file_name':file, 'img_bytes':img_bytes}
                except Exception as e:
                    print [f'[WARNING] Error when processing file: {e}']
            
    # Number of images
    n_images = len(data_dict)
    print (f'[INFO] Completed. Number of images: {n_images}')

    return data_dict


data_dict = create_input_data_dict(input_dir)

[INFO] Running on Data Source...
[INFO] Completed. Number of images: 2172


## Select and Open 2 Random Images

In [4]:
def get_2_rand_images (data_dict):
    
    # List of hashes (keys in data_dict)
    hash_list = list(data_dict.keys())
    # File 1
    hash_1 = random.choice(hash_list)
    file_path_1 = data_dict[hash_1]['file_path']
    file_name_1 = data_dict[hash_1]['file_name']
    img_bytes_1 = data_dict[hash_1]['img_bytes']
    # File 2
    hash_2 = random.choice(hash_list)
    file_path_2 = data_dict[hash_2]['file_path']
    file_name_2 = data_dict[hash_2]['file_name']
    img_bytes_2 = data_dict[hash_2]['img_bytes']

    # # Image 1 bytes serialization
    # img_byte_1 = io.BytesIO()
    # img_1.save(img_byte_1, format = img_1.format)
    # img_byte_1.seek(0)
    # img_data_1 = img_byte_1.read()

    # # Image 1 bytes serialization
    # img_byte_2 = io.BytesIO()
    # img_2.save(img_byte_2, format = img_2.format)
    # img_byte_2.seek(0)
    # img_data_2 = img_byte_2.read()

    return (file_path_1, file_name_1, img_bytes_1), (file_path_2, file_name_2, img_bytes_2)

## Widgets (Start the App)

In [ ]:
# Current Step
n = 1

# Initial Images
(file_path_1, file_name_1, img_bytes_1), (file_path_2, file_name_2, img_bytes_2) = get_2_rand_images(data_dict)
img_widget_1 = widgets.Image(value=img_bytes_1, format='jpg', width=300, height=400)
img_widget_2 = widgets.Image(value=img_bytes_2, format='jpg', width=300, height=400)

# Title label
lbl_title_value = f'Quality Model App'
lbl_title = widgets.HTML(value=f'<p style="font-size: 24px ; font-weight: bold ; color:rgb(75,75,75)">{lbl_title_value}</p>')

# Tagging User
lbl_user = widgets.HTML(value=f'<p style="font-size: 16px ; font-weight: bold ; color:rgb(75,75,75) ; width: 120px ; height: 20px">Tagging User</p>')
txt_user = widgets.Text(value='', disabled=False)

# Task String User
lbl_task = widgets.HTML(value=f'<p style="font-size: 16px ; font-weight: bold ; color:rgb(75,75,75) ; width: 120px ; height: 20px">Task String</p>')
txt_task = widgets.Text(value='', disabled=False)

# Status label
lbl_status_value = f'Choose Best Image - {n}'
lbl_status = widgets.HTML(value=f'<p style="font-size: 20px ; font-weight: bold ; color:rgb(75,75,75)">{lbl_status_value}</p>')

# Selection buttons
btn_select_1 = widgets.Button(description = 'SELECT', icon='check', button_style = 'success')
btn_select_1.style.button_color = 'rgb(30,144,255)'
btn_select_2 = widgets.Button(description = 'SELECT', icon='check', button_style = 'success')
btn_select_2.style.button_color = 'rgb(30,144,255)'

# Skip button
btn_skip = widgets.Button(description = 'SKIP')
btn_skip.style.button_color = 'rgb(225,225,225)'

# Layout
box_layout = widgets.Layout(display='flex',
                            flex_flow='row',
                            justify_content = 'space-around',
                            align_items='center',
                            width='100%'
                            )

def show_widgets(lbl_title, lbl_status, lbl_user, txt_user, lbl_task, txt_task, img_1, img_2, btn_select_1, btn_select_2, btn_skip, box_layout):
    box_title = widgets.Box(children=[lbl_title], layout=widgets.Layout(display='flex', flex_flow='row', justify_content = 'flex-start', align_items='center', width='100%'))
    box_user = widgets.Box(children=[lbl_user, txt_user], layout=widgets.Layout(display='flex', flex_flow='row', justify_content = 'flex-start', align_items='center', width='100%'))
    box_task = widgets.Box(children=[lbl_task, txt_task], layout=widgets.Layout(display='flex', flex_flow='row', justify_content = 'flex-start', align_items='center', width='100%'))
    box_user_tax = widgets.Box(children=[box_user, box_task], layout=widgets.Layout(display='flex', flex_flow='row', justify_content = 'flex-start', align_items='center', width='100%'))
    box_status = widgets.Box(children=[lbl_status], layout=widgets.Layout(display='flex', flex_flow='row', justify_content = 'flex-start', align_items='center', width='100%'))
    box_images = widgets.Box(children=[img_1, img_2], layout=box_layout)
    box_select = widgets.Box(children=[btn_select_1, btn_select_2], layout=box_layout)
    box_skip = widgets.Box(children=[btn_skip], layout=box_layout)
    display(box_title)
    display(box_user_tax)
    #display(box_task)
    display(box_status)
    display(box_images)
    display(box_select)
    display(box_skip)

def skip_pressed(button):
    # Increment step
    global n
    n += 1
    clear_output()
    # Update status label
    lbl_status_value = f'Choose Best Image - {n}'
    lbl_status = widgets.HTML(value=f'<p style="font-size: 20px ; font-weight: bold ; color:rgb(75,75,75)">{lbl_status_value}</p>')
    # Get new images
    (file_path_1, file_name_1, img_bytes_1), (file_path_2, file_name_2, img_bytes_2) = get_2_rand_images(data_dict)
    img_widget_1 = widgets.Image(value=img_bytes_1, format='jpg', width=300, height=400)
    img_widget_2 = widgets.Image(value=img_bytes_2, format='jpg', width=300, height=400)
    show_widgets(lbl_title, lbl_status, lbl_user, txt_user, lbl_task, txt_task, img_widget_1, img_widget_2, btn_select_1, btn_select_2, btn_skip, box_layout)

def select_pressed(button):
    # Increment step
    global n
    n += 1
    clear_output()
    # Update title label
    lbl_status_value = f'Choose Best Image - {n}'
    lbl_status = widgets.HTML(value=f'<p style="font-size: 20px ; font-weight: bold ; color:rgb(75,75,75)">{lbl_status_value}</p>')
    # Get new images
    (file_path_1, file_name_1, img_bytes_1), (file_path_2, file_name_2, img_bytes_2) = get_2_rand_images(data_dict)
    img_widget_1 = widgets.Image(value=img_bytes_1, format='jpg', width=300, height=400)
    img_widget_2 = widgets.Image(value=img_bytes_2, format='jpg', width=300, height=400)
    show_widgets(lbl_title, lbl_status, lbl_user, txt_user, lbl_task, txt_task, img_widget_1, img_widget_2, btn_select_1, btn_select_2, btn_skip, box_layout)

# binding skip button to skip function callback
btn_skip.on_click(skip_pressed)

# binding select button 1 and 2 to select function callback
btn_select_1.on_click(select_pressed)
btn_select_2.on_click(select_pressed)

show_widgets(lbl_title, lbl_status, lbl_user, txt_user, lbl_task, txt_task, img_widget_1, img_widget_2, btn_select_1, btn_select_2, btn_skip, box_layout)

Box(children=(HTML(value='<p style="font-size: 24px ; font-weight: bold ; color:rgb(75,75,75)">Quality Model A…

Box(children=(Box(children=(HTML(value='<p style="font-size: 16px ; font-weight: bold ; color:rgb(75,75,75) ; …

Box(children=(HTML(value='<p style="font-size: 20px ; font-weight: bold ; color:rgb(75,75,75)">Choose Best Ima…

Box(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00 \x00\x00\x00 \x00\x08\x06\x00\x00\x00r…

Box(children=(Button(button_style='success', description='SELECT', icon='check', style=ButtonStyle(button_colo…

Box(children=(Button(description='SKIP', style=ButtonStyle(button_color='rgb(225,225,225)')),), layout=Layout(…